In [41]:
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin

In [42]:
df = pd.read_csv('../../../../raw_data/data_prep.csv')

In [43]:
train, dev = train_test_split(df, test_size=0.1, random_state=42)

In [44]:
train.shape, dev.shape

((1080, 8), (120, 8))

In [45]:
df['rating'].value_counts()

AAA     153
A-      150
A+      126
BBB+    115
A       110
BBB      90
AA       74
AA+      71
BBB-     65
BB+      57
AA-      54
BB       50
BB-      29
B+       20
B        12
B-       12
C        12
Name: rating, dtype: int64

In [46]:
categories = ['AAA', 'AA+', 'AA-', 'AA']

In [47]:
dev = dev.query('rating in @categories')
train = train.query('rating in @categories')

In [48]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [49]:
train.shape, dev.shape

((312, 8), (40, 8))

In [50]:
nlp = spacy.load('ru_core_news_lg')

In [51]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["rating"][i]] = 1
        db.add(doc)
    return db

In [54]:
create_docbin(dev).to_disk("dev.spacy")
create_docbin(train).to_disk("train.spacy")

In [55]:
! python -m spacy train config.cfg --output ./subsym_model --paths.train train.spacy --paths.dev dev.spacy

ℹ Saving to output directory: subsym_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       52.60    0.53
  0     200          32.12       86.69    0.87
  1     400          18.32       87.34    0.87
  1     600           8.83       89.55    0.90
  2     800           4.23       90.92    0.91
  3    1000           3.60       90.76    0.91
  3    1200           3.41       91.05    0.91
  4    1400           2.24       90.54    0.91
  5    1600           2.56       90.09    0.90
  5    1800           1.26       90.06    0.90
  6    2000           0.95       90.51    0.91
  7    2200           1.62       90.30    0.90
  7    2400           1.56       90.00    0